# CLIP 对比学习机制深度实验：从 InfoNCE 到 SigLIP

## 实验目标

1. **从零实现 InfoNCE 损失**并分析其梯度行为——理解对比学习的优化本质
2. **温度参数 τ 实验**——可视化 τ 如何控制 softmax 锐度和梯度分布
3. **构建 MiniCLIP**（手写 Attention）并在 CIFAR-10 上训练，观察相似度矩阵的演化
4. **消融实验**：对称 vs 非对称损失、Batch Size 对负样本质量的影响
5. **Embedding 空间分析**：用 Alignment & Uniformity 度量评估表示质量
6. **SigLIP 对比**：实现 Sigmoid 损失替代 Softmax，比较两种对比学习范式

## 预期结果

- InfoNCE 手写版与 `F.cross_entropy` 版数值一致（误差 < 1e-5）
- 低温度 τ 时 softmax 分布趋近 one-hot（熵低），高 τ 时趋近均匀分布（熵高）
- 训练过程中相似度矩阵对角线逐渐变亮，off-diagonal 逐渐变暗
- 对称损失零样本准确率 ≥ 单向损失
- 大 batch（更多负样本）的零样本准确率 > 小 batch
- 训练后模型的 Alignment ↓、Uniformity ↓（优于随机初始化）
- SigLIP 也能学到有意义的表示（准确率 > 10% 随机基线）

## 所需环境

- Python >= 3.9
- PyTorch >= 2.0
- torchvision
- matplotlib
- numpy

## 关联笔记

- [对比学习详解](../../notes/fundamentals/contrastive-learning.md) — InfoNCE 推导、温度分析、SigLIP 理论
- [CLIP 论文笔记](../../papers/clip.md) — 论文细节、prompt engineering 策略
- [多模态模型发展](../../notes/multimodal-arch/mllm-evolution.md) — CLIP 在 MLLM 中的角色
- [MiniCLIP + FSDP 实验](./multimodal_pretrain_fsdp.ipynb) — 基本实现（本 notebook 聚焦机制分析）

In [ ]:
# ======== Part 1: 环境设置与工具函数 ========
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
import math
import time
import random
from torch.utils.data import Dataset, DataLoader


def set_seed(seed=42):
    """固定随机种子，确保实验可复现"""
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)


set_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'使用设备: {device}')
print(f'PyTorch 版本: {torch.__version__}')


# ======== 工具函数：后续多处复用 ========

def cosine_sim_matrix(A, B):
    """计算两组 L2 归一化向量间的余弦相似度矩阵
    Args:
        A: (N, D) 已归一化的向量
        B: (M, D) 已归一化的向量
    Returns:
        (N, M) 余弦相似度矩阵
    """
    return A @ B.T


def plot_similarity_matrix(matrix, title='', xlabel='Text', ylabel='Image', cmap='Blues'):
    """绘制相似度矩阵热力图（本 notebook 中多次使用）"""
    fig, ax = plt.subplots(figsize=(6, 5))
    im = ax.imshow(matrix, cmap=cmap, vmin=-1 if matrix.min() < 0 else 0, vmax=1)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    plt.colorbar(im, ax=ax, shrink=0.8)
    plt.tight_layout()
    return fig, ax


print('✓ Part 1 设置完成')

## Part 2: InfoNCE 损失函数——从零实现与梯度分析

InfoNCE（Noise-Contrastive Estimation）来自互信息最大化的下界：

$$I(X; Y) \geq \log K - \mathcal{L}_{\text{InfoNCE}}$$

其中 $K$ 是 batch size（负样本数量 + 1），$\mathcal{L}_{\text{InfoNCE}}$ 为：

$$\mathcal{L}_{\text{InfoNCE}} = -\frac{1}{N} \sum_{i=1}^{N} \log \frac{\exp(\text{sim}(z_i^{\text{img}}, z_i^{\text{txt}}) / \tau)}{\sum_{k=1}^{N} \exp(\text{sim}(z_i^{\text{img}}, z_k^{\text{txt}}) / \tau)}$$

**关键洞察**：InfoNCE 本质上等价于一个 N 分类的 cross-entropy——标签就是对角线索引。

CLIP 使用**对称版本**：$\mathcal{L} = \frac{1}{2}(\mathcal{L}_{i \to t} + \mathcal{L}_{t \to i})$，
同时优化 image→text 和 text→image 两个方向。

下面我们从零实现这个损失函数，并**可视化其梯度**——观察优化信号如何分布在正样本和负样本上。

In [ ]:
# ======== Part 2: InfoNCE 损失——从零实现与梯度分析 ========

def info_nce_loss_manual(sim_matrix, tau=0.07):
    """
    手写 InfoNCE 损失（不使用 F.cross_entropy）
    
    Args:
        sim_matrix: (N, N) 余弦相似度矩阵，对角线为正样本对
        tau: 温度参数
    Returns:
        symmetric_loss: 对称 InfoNCE 损失
    """
    N = sim_matrix.size(0)
    scaled = sim_matrix / tau  # (N, N)
    
    # Image → Text 方向：每行做 softmax，标签为该行索引
    # log_softmax(scaled)[i, i] = scaled[i,i] - log(sum_k exp(scaled[i,k]))
    loss_i2t = 0.0
    for i in range(N):
        log_sum_exp = torch.logsumexp(scaled[i], dim=0)
        loss_i2t += -(scaled[i, i] - log_sum_exp)
    loss_i2t = loss_i2t / N
    
    # Text → Image 方向：每列做 softmax，标签为该列索引
    loss_t2i = 0.0
    for j in range(N):
        log_sum_exp = torch.logsumexp(scaled[:, j], dim=0)
        loss_t2i += -(scaled[j, j] - log_sum_exp)
    loss_t2i = loss_t2i / N
    
    return (loss_i2t + loss_t2i) / 2


def info_nce_loss_efficient(sim_matrix, tau=0.07):
    """
    高效版 InfoNCE：利用 F.cross_entropy 的等价性
    
    InfoNCE 等价于 N 分类 cross-entropy，标签为 [0, 1, 2, ..., N-1]
    """
    N = sim_matrix.size(0)
    labels = torch.arange(N, device=sim_matrix.device)
    scaled = sim_matrix / tau
    
    loss_i2t = F.cross_entropy(scaled, labels)        # 行方向
    loss_t2i = F.cross_entropy(scaled.T, labels)      # 列方向
    
    return (loss_i2t + loss_t2i) / 2


# ======== 验证两种实现的一致性 ========
set_seed(42)
N = 8
# 生成随机归一化 embedding，计算余弦相似度
img_emb = F.normalize(torch.randn(N, 64), dim=-1)
txt_emb = F.normalize(torch.randn(N, 64), dim=-1)
sim = cosine_sim_matrix(img_emb, txt_emb)

loss_manual = info_nce_loss_manual(sim, tau=0.07)
loss_efficient = info_nce_loss_efficient(sim, tau=0.07)
diff = (loss_manual - loss_efficient).abs().item()

print(f'手写版 Loss: {loss_manual.item():.6f}')
print(f'高效版 Loss: {loss_efficient.item():.6f}')
print(f'差异: {diff:.2e}')
assert diff < 1e-5, f'两版实现不一致！差异={diff}'
print('✓ InfoNCE 两种实现验证一致')


# ======== 梯度分析：∂L/∂sim 的分布 ========
set_seed(42)
sim_grad = cosine_sim_matrix(
    F.normalize(torch.randn(8, 64), dim=-1),
    F.normalize(torch.randn(8, 64), dim=-1)
).requires_grad_(True)

loss = info_nce_loss_efficient(sim_grad, tau=0.07)
loss.backward()

grad = sim_grad.grad.detach()

fig, axes = plt.subplots(1, 3, figsize=(16, 4.5))

# (a) 相似度矩阵
im0 = axes[0].imshow(sim_grad.detach().numpy(), cmap='RdBu_r', vmin=-1, vmax=1)
axes[0].set_title('余弦相似度矩阵 $S_{ij}$')
axes[0].set_xlabel('Text 索引')
axes[0].set_ylabel('Image 索引')
plt.colorbar(im0, ax=axes[0], shrink=0.8)

# (b) 梯度绝对值
im1 = axes[1].imshow(grad.abs().numpy(), cmap='Reds')
axes[1].set_title('梯度绝对值 $|\\partial L / \\partial S_{ij}|$')
axes[1].set_xlabel('Text 索引')
axes[1].set_ylabel('Image 索引')
plt.colorbar(im1, ax=axes[1], shrink=0.8)

# (c) 对角线 vs 非对角线梯度分布
diag_grad = grad.diag().numpy()
offdiag_grad = grad[~torch.eye(8, dtype=bool)].numpy()
axes[2].hist(diag_grad, bins=8, alpha=0.7, label='正样本对（对角线）', color='green')
axes[2].hist(offdiag_grad, bins=15, alpha=0.7, label='负样本对（非对角线）', color='red')
axes[2].set_title('梯度值分布')
axes[2].set_xlabel('$\\partial L / \\partial S_{ij}$')
axes[2].set_ylabel('计数')
axes[2].legend(fontsize=9)
axes[2].axvline(x=0, color='black', linestyle='--', alpha=0.3)

plt.suptitle('InfoNCE 损失的梯度分析', fontsize=14)
plt.tight_layout()
plt.show()

print('观察：')
print('- 正样本对（对角线）的梯度为负值 → 优化方向是增大正样本相似度')
print('- 负样本对的梯度为正值 → 优化方向是减小负样本相似度')
print(f'- 正样本平均梯度: {diag_grad.mean():.4f}, 负样本平均梯度: {offdiag_grad.mean():.4f}')

## Part 3: 温度参数 τ 的作用

温度参数 $\tau$ 控制 softmax 的锐度：

$$p_k = \frac{\exp(s_k / \tau)}{\sum_j \exp(s_j / \tau)}$$

- **$\tau \to 0$**：分布趋近 one-hot，只关注最困难的负样本（hard negative mining），但梯度可能不稳定
- **$\tau \to \infty$**：分布趋近均匀，所有负样本权重相等，判别能力弱
- **$\tau \approx 0.07$**：CLIP 的最佳温度，平衡锐度与稳定性

CLIP 使用**可学习温度**：$\text{logit\_scale} = \ln(1/\tau)$，初始化为 $\ln(1/0.07) \approx 2.66$，
并裁剪 $\exp(\text{logit\_scale}) \leq 100$（即 $\tau \geq 0.01$）。

下面我们通过实验验证温度对分布形态和梯度幅度的影响。

In [ ]:
# ======== Part 3: 温度参数 τ 对 softmax 分布的影响 ========

set_seed(42)

# 构造一个 16-way 的相似度向量（1 个正样本 + 15 个负样本）
N_candidates = 16
# 正样本相似度较高(0.6)，负样本相似度分散在 [-0.1, 0.4]
sims = torch.tensor([0.6] + list(np.random.uniform(-0.1, 0.4, N_candidates - 1)), dtype=torch.float32)

tau_values = [0.01, 0.05, 0.07, 0.1, 0.3, 0.5, 1.0, 2.0]

# 收集各温度下的统计量
entropies = []
grad_norms = []
all_probs = []

for tau in tau_values:
    # softmax 概率分布
    probs = F.softmax(sims / tau, dim=0)
    all_probs.append(probs.numpy())
    
    # 熵: H = -sum(p * log(p))
    entropy = -(probs * torch.log(probs + 1e-10)).sum().item()
    entropies.append(entropy)
    
    # 梯度范数
    sims_g = sims.clone().requires_grad_(True)
    loss = -torch.log(F.softmax(sims_g / tau, dim=0)[0])  # 正样本在索引 0
    loss.backward()
    grad_norms.append(sims_g.grad.norm().item())

# ======== 三面板可视化 ========
fig, axes = plt.subplots(1, 3, figsize=(17, 5))

# (a) 不同 τ 下的概率分布
for i, tau in enumerate(tau_values):
    if tau in [0.01, 0.07, 0.5, 2.0]:  # 选几个代表性的
        axes[0].plot(all_probs[i], 'o-', label=f'τ={tau}', markersize=4, alpha=0.8)
axes[0].axhline(y=1/N_candidates, color='gray', linestyle='--', alpha=0.5, label=f'均匀分布 (1/{N_candidates})')
axes[0].set_xlabel('候选样本索引（0=正样本）')
axes[0].set_ylabel('softmax 概率')
axes[0].set_title('(a) τ 对 softmax 分布形态的影响')
axes[0].legend(fontsize=8)

# (b) 熵 vs τ
axes[1].plot(tau_values, entropies, 'bs-', linewidth=2, markersize=6)
axes[1].axhline(y=np.log(N_candidates), color='gray', linestyle='--', alpha=0.5, label=f'最大熵 ln({N_candidates})={np.log(N_candidates):.2f}')
axes[1].set_xlabel('温度 τ')
axes[1].set_ylabel('分布熵 H')
axes[1].set_title('(b) 温度与分布熵的关系')
axes[1].set_xscale('log')
axes[1].legend(fontsize=9)

# (c) 梯度范数 vs τ
axes[2].plot(tau_values, grad_norms, 'r^-', linewidth=2, markersize=6)
axes[2].set_xlabel('温度 τ')
axes[2].set_ylabel('梯度 L2 范数')
axes[2].set_title('(c) 温度与梯度幅度的关系')
axes[2].set_xscale('log')

plt.suptitle('温度参数 τ 的三重影响', fontsize=14)
plt.tight_layout()
plt.show()

# 验证
assert entropies[0] < entropies[-1], '低温度应该有更低的熵'
print('✓ 低温度 τ 的熵 < 高温度 τ 的熵')
print(f'\nτ=0.01 → 正样本概率={all_probs[0][0]:.4f}, 熵={entropies[0]:.4f}（接近 one-hot）')
print(f'τ=0.07 → 正样本概率={all_probs[2][0]:.4f}, 熵={entropies[2]:.4f}（CLIP 默认）')
print(f'τ=2.00 → 正样本概率={all_probs[-1][0]:.4f}, 熵={entropies[-1]:.4f}（接近均匀）')
print(f'\n结论：τ≈0.07 在锐度和梯度稳定性之间取得了好的平衡')

## Part 4: MiniCLIP 模型——模块化实现

CLIP 由两个独立的编码器组成，通过线性投影将各自的表示映射到共享的 embedding 空间：

```
Image ──→ [Patch Embed] ──→ [ViT Blocks ×4] ──→ [CLS token] ──→ [Linear Proj] ──→ img_embed
                                                                                       ↕ cosine sim
Text  ──→ [Token Embed] ──→ [TF Blocks ×3]  ──→ [Pool]      ──→ [Linear Proj] ──→ txt_embed
```

与现有 `multimodal_pretrain_fsdp.ipynb` 的差异：
- **更小的模型**（d=192, V4+T3 vs d=256, V6+T4）以便快速消融
- **手写 Attention**（非 `nn.MultiheadAttention`）以便检查注意力权重
- **模块化设计**：每个组件独立可测试

In [ ]:
# ======== Part 4: MiniCLIP 模型——模块化实现 ========

# ---- Patch Embedding ----
class PatchEmbed(nn.Module):
    """将图像分割为 patch 并嵌入到 d_model 维空间"""
    def __init__(self, img_size=32, patch_size=4, in_channels=3, embed_dim=192):
        super().__init__()
        self.num_patches = (img_size // patch_size) ** 2  # 32/4=8, 8*8=64 patches
        self.proj = nn.Conv2d(in_channels, embed_dim, kernel_size=patch_size, stride=patch_size)
        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        self.pos_embed = nn.Parameter(torch.zeros(1, self.num_patches + 1, embed_dim))  # +1 for CLS
        nn.init.normal_(self.cls_token, std=0.01)
        nn.init.normal_(self.pos_embed, std=0.01)
    
    def forward(self, x):
        """x: (B, 3, H, W) -> (B, num_patches+1, embed_dim)"""
        B = x.shape[0]
        x = self.proj(x)                          # (B, embed_dim, H/P, W/P)
        x = x.flatten(2).transpose(1, 2)          # (B, num_patches, embed_dim)
        cls = self.cls_token.expand(B, -1, -1)    # (B, 1, embed_dim)
        x = torch.cat([cls, x], dim=1)            # (B, num_patches+1, embed_dim)
        x = x + self.pos_embed
        return x


# ---- 手写 Multi-Head Self-Attention ----
class Attention(nn.Module):
    """手写 Attention，便于后续提取注意力权重"""
    def __init__(self, dim, n_heads=6):
        super().__init__()
        assert dim % n_heads == 0
        self.n_heads = n_heads
        self.d_k = dim // n_heads
        self.W_qkv = nn.Linear(dim, dim * 3, bias=False)  # 合并 Q/K/V 投影
        self.W_o = nn.Linear(dim, dim, bias=False)
    
    def forward(self, x, mask=None):
        """x: (B, T, D) -> output: (B, T, D), attn_weights: (B, H, T, T)"""
        B, T, D = x.shape
        qkv = self.W_qkv(x).reshape(B, T, 3, self.n_heads, self.d_k)
        qkv = qkv.permute(2, 0, 3, 1, 4)  # (3, B, H, T, d_k)
        Q, K, V = qkv[0], qkv[1], qkv[2]
        
        scores = (Q @ K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            scores = scores.masked_fill(mask.bool(), float('-inf'))
        
        attn_weights = F.softmax(scores, dim=-1)  # (B, H, T, T)
        out = (attn_weights @ V).transpose(1, 2).reshape(B, T, D)
        return self.W_o(out), attn_weights


# ---- Transformer Block (Pre-LN) ----
class Block(nn.Module):
    def __init__(self, dim, n_heads=6, mlp_ratio=4.0):
        super().__init__()
        self.norm1 = nn.LayerNorm(dim)
        self.attn = Attention(dim, n_heads)
        self.norm2 = nn.LayerNorm(dim)
        hidden = int(dim * mlp_ratio)
        self.mlp = nn.Sequential(
            nn.Linear(dim, hidden),
            nn.GELU(),
            nn.Linear(hidden, dim)
        )
    
    def forward(self, x, mask=None):
        attn_out, attn_w = self.attn(self.norm1(x), mask=mask)
        x = x + attn_out
        x = x + self.mlp(self.norm2(x))
        return x, attn_w


# ---- Image Encoder (ViT) ----
class ImageEncoder(nn.Module):
    def __init__(self, img_size=32, patch_size=4, embed_dim=192, n_layers=4, n_heads=6):
        super().__init__()
        self.patch_embed = PatchEmbed(img_size, patch_size, 3, embed_dim)
        self.blocks = nn.ModuleList([Block(embed_dim, n_heads) for _ in range(n_layers)])
        self.norm = nn.LayerNorm(embed_dim)
    
    def forward(self, x):
        """x: (B, 3, H, W) -> (B, embed_dim) CLS token"""
        x = self.patch_embed(x)
        for block in self.blocks:
            x, _ = block(x)
        x = self.norm(x)
        return x[:, 0]  # CLS token


# ---- Text Encoder ----
class TextEncoder(nn.Module):
    def __init__(self, vocab_size=256, embed_dim=192, max_len=32, n_layers=3, n_heads=6):
        super().__init__()
        self.token_emb = nn.Embedding(vocab_size, embed_dim)
        self.pos_emb = nn.Parameter(torch.zeros(1, max_len, embed_dim))
        nn.init.normal_(self.pos_emb, std=0.01)
        self.blocks = nn.ModuleList([Block(embed_dim, n_heads) for _ in range(n_layers)])
        self.norm = nn.LayerNorm(embed_dim)
    
    def forward(self, tokens, mask=None):
        """tokens: (B, T) -> (B, embed_dim) 平均池化"""
        x = self.token_emb(tokens) + self.pos_emb[:, :tokens.size(1)]
        for block in self.blocks:
            x, _ = block(x)
        x = self.norm(x)
        # 平均池化（忽略 padding=0 的位置）
        if mask is not None:
            mask_expanded = mask.unsqueeze(-1).float()  # (B, T, 1)
            x = (x * mask_expanded).sum(dim=1) / mask_expanded.sum(dim=1).clamp(min=1)
        else:
            x = x.mean(dim=1)
        return x


# ---- 完整 MiniCLIP ----
class MiniCLIP(nn.Module):
    def __init__(self, embed_dim=192, proj_dim=128, img_size=32, patch_size=4,
                 v_layers=4, t_layers=3, n_heads=6, vocab_size=256, max_len=32):
        super().__init__()
        self.image_encoder = ImageEncoder(img_size, patch_size, embed_dim, v_layers, n_heads)
        self.text_encoder = TextEncoder(vocab_size, embed_dim, max_len, t_layers, n_heads)
        
        # 投影到共享 embedding 空间
        self.image_proj = nn.Linear(embed_dim, proj_dim, bias=False)
        self.text_proj = nn.Linear(embed_dim, proj_dim, bias=False)
        
        # 可学习温度参数: logit_scale = ln(1/τ)
        self.logit_scale = nn.Parameter(torch.tensor(np.log(1 / 0.07), dtype=torch.float32))
    
    def encode_image(self, images):
        """images: (B, 3, H, W) -> (B, proj_dim) L2 归一化"""
        feat = self.image_encoder(images)
        return F.normalize(self.image_proj(feat), dim=-1)
    
    def encode_text(self, tokens, mask=None):
        """tokens: (B, T) -> (B, proj_dim) L2 归一化"""
        feat = self.text_encoder(tokens, mask)
        return F.normalize(self.text_proj(feat), dim=-1)
    
    def forward(self, images, tokens, text_mask=None):
        """返回 (logits_per_image, img_embed, txt_embed)"""
        img_embed = self.encode_image(images)
        txt_embed = self.encode_text(tokens, text_mask)
        
        # 余弦相似度 × 温度缩放
        logit_scale = self.logit_scale.exp().clamp(max=100.0)
        logits = logit_scale * cosine_sim_matrix(img_embed, txt_embed)
        
        return logits, img_embed, txt_embed


# ======== 验证 ========
set_seed(42)
model = MiniCLIP().to(device)

# 测试前向传播
dummy_img = torch.randn(2, 3, 32, 32).to(device)
dummy_txt = torch.randint(1, 100, (2, 16)).to(device)
logits, img_e, txt_e = model(dummy_img, dummy_txt)

assert img_e.shape == (2, 128), f'img_embed 形状错误: {img_e.shape}'
assert txt_e.shape == (2, 128), f'txt_embed 形状错误: {txt_e.shape}'
assert logits.shape == (2, 2), f'logits 形状错误: {logits.shape}'

# L2 归一化验证
assert (img_e.norm(dim=-1) - 1.0).abs().max() < 1e-5, 'img_embed 未归一化'
assert (txt_e.norm(dim=-1) - 1.0).abs().max() < 1e-5, 'txt_embed 未归一化'

# 参数量分组统计
def count_params(module):
    return sum(p.numel() for p in module.parameters())

print(f'MiniCLIP 参数量:')
print(f'  Image Encoder: {count_params(model.image_encoder):>10,}')
print(f'  Text Encoder:  {count_params(model.text_encoder):>10,}')
print(f'  Image Proj:    {count_params(model.image_proj):>10,}')
print(f'  Text Proj:     {count_params(model.text_proj):>10,}')
print(f'  总计:          {count_params(model):>10,}')
print(f'\n初始温度: τ = {(1/model.logit_scale.exp()).item():.4f}')
print('✓ MiniCLIP 前向传播验证通过')

## Part 5: 数据准备——CIFAR-10 + 多样化 Caption

CLIP 原文使用 4 亿 image-text pairs 从互联网爬取。我们使用 CIFAR-10 作为代理，
通过**模板生成**创建 caption。

CLIP 论文的一个重要发现：零样本分类时使用 **prompt ensemble**（80 个模板取平均）
比单个模板 `"a photo of a {class}"` 高 **+4.8%** 准确率。
我们设计 12 个模板，后续实验将验证模板多样性的影响。

In [ ]:
# ======== Part 5: 数据准备——CIFAR-10 + 多样化 Caption ========

CIFAR10_CLASSES = [
    'airplane', 'automobile', 'bird', 'cat', 'deer',
    'dog', 'frog', 'horse', 'ship', 'truck'
]

# 12 种 caption 模板（比现有 notebook 的 8 种更丰富）
CAPTION_TEMPLATES = [
    'a photo of a {}',
    'a picture of a {}',
    'a {} in the scene',
    'an image showing a {}',
    'a small {} in the photo',
    'this is a {}',
    'a blurry photo of a {}',
    'a close-up photo of a {}',
    'a bright photo of a {}',
    'a dark photo of a {}',
    'a drawing of a {}',
    'a {} on display',
]


class SimpleTokenizer:
    """字符级 tokenizer（简单但足够演示 CLIP 机制）"""
    def __init__(self, max_len=32):
        self.max_len = max_len
        # ASCII 可打印字符 + PAD(0) + UNK(1)
        self.vocab = {chr(i): i - 30 for i in range(32, 127)}  # ' '=2, '!'=3, ...
        self.vocab['<PAD>'] = 0
        self.vocab['<UNK>'] = 1
        self.inv_vocab = {v: k for k, v in self.vocab.items()}
        self.vocab_size = len(self.vocab)
    
    def encode(self, text):
        """text -> token ids (with padding)"""
        tokens = [self.vocab.get(c, 1) for c in text.lower()[:self.max_len]]
        # Padding
        tokens = tokens + [0] * (self.max_len - len(tokens))
        return tokens
    
    def decode(self, tokens):
        """token ids -> text (去掉 padding)"""
        chars = [self.inv_vocab.get(t, '?') for t in tokens if t != 0]
        return ''.join(chars)


tokenizer = SimpleTokenizer(max_len=32)
print(f'词表大小: {tokenizer.vocab_size}')
print(f'编码示例: "{"a photo of a cat"}" → {tokenizer.encode("a photo of a cat")[:20]}...')
print(f'解码还原: {tokenizer.decode(tokenizer.encode("a photo of a cat"))}')


class CIFAR10CaptionDataset(Dataset):
    """CIFAR-10 + 模板生成的 caption"""
    def __init__(self, cifar_dataset, tokenizer, templates=CAPTION_TEMPLATES):
        self.cifar = cifar_dataset
        self.tokenizer = tokenizer
        self.templates = templates
    
    def __len__(self):
        return len(self.cifar)
    
    def __getitem__(self, idx):
        image, label = self.cifar[idx]
        # 随机选模板，增加多样性
        template = random.choice(self.templates)
        caption = template.format(CIFAR10_CLASSES[label])
        tokens = torch.tensor(self.tokenizer.encode(caption), dtype=torch.long)
        text_mask = (tokens != 0).float()  # padding mask
        return image, tokens, text_mask, label


# 数据增强和加载
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616)),
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616)),
])

cifar_train = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
cifar_test = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_dataset = CIFAR10CaptionDataset(cifar_train, tokenizer)
test_dataset = CIFAR10CaptionDataset(cifar_test, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False, num_workers=2)

print(f'\n训练集: {len(train_dataset)} 张图片')
print(f'测试集: {len(test_dataset)} 张图片')
print(f'类别数: {len(CIFAR10_CLASSES)}')
print(f'Caption 模板数: {len(CAPTION_TEMPLATES)}')

# 可视化样本
fig, axes = plt.subplots(2, 5, figsize=(14, 5))
inv_norm = transforms.Normalize(
    mean=[-0.4914/0.2470, -0.4822/0.2435, -0.4465/0.2616],
    std=[1/0.2470, 1/0.2435, 1/0.2616]
)
for i in range(10):
    img, tokens, mask, label = train_dataset[i * 5000]
    ax = axes[i // 5, i % 5]
    img_display = inv_norm(img).permute(1, 2, 0).clamp(0, 1).numpy()
    ax.imshow(img_display)
    caption = tokenizer.decode(tokens.tolist())
    ax.set_title(caption, fontsize=8)
    ax.axis('off')

plt.suptitle('CIFAR-10 样本及生成的 Caption', fontsize=13)
plt.tight_layout()
plt.show()

assert len(train_dataset) == 50000
print('✓ 数据准备完成')

## Part 6: 训练 MiniCLIP + 相似度矩阵可视化

训练目标：对称 InfoNCE 损失

$$\mathcal{L} = \frac{1}{2} \left[ \text{CE}(\text{logits}, \text{labels}) + \text{CE}(\text{logits}^T, \text{labels}) \right]$$

**核心可视化**：训练过程中的相似度矩阵快照——一个训练良好的 CLIP 模型应该展现出
"对角线逐渐变亮、非对角线逐渐变暗"的模式。

In [ ]:
# ======== Part 6: 训练——带相似度矩阵实时可视化 ========

def clip_loss(logits):
    """对称 InfoNCE 损失"""
    N = logits.size(0)
    labels = torch.arange(N, device=logits.device)
    loss_i2t = F.cross_entropy(logits, labels)
    loss_t2i = F.cross_entropy(logits.T, labels)
    return (loss_i2t + loss_t2i) / 2


def train_clip(model, train_loader, n_epochs=15, lr=5e-4, device='cpu',
               loss_fn=clip_loss, snapshot_epochs=None):
    """
    训练 CLIP 模型
    
    Args:
        snapshot_epochs: 在这些 epoch 保存相似度矩阵快照
    Returns:
        loss_history, sim_snapshots
    """
    model = model.to(device)
    model.train()
    
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=0.05, betas=(0.9, 0.98))
    total_steps = n_epochs * len(train_loader)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=total_steps, eta_min=lr * 0.1)
    
    if snapshot_epochs is None:
        snapshot_epochs = set()
    
    loss_history = []
    sim_snapshots = {}  # epoch -> similarity matrix
    
    for epoch in range(n_epochs):
        epoch_loss = 0
        for batch_idx, (images, tokens, text_mask, _) in enumerate(train_loader):
            images = images.to(device)
            tokens = tokens.to(device)
            text_mask = text_mask.to(device)
            
            logits, img_emb, txt_emb = model(images, tokens, text_mask)
            loss = loss_fn(logits)
            
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            scheduler.step()
            
            epoch_loss += loss.item()
            loss_history.append(loss.item())
            
            # 保存第一个 batch 的相似度矩阵快照
            if batch_idx == 0 and (epoch + 1) in snapshot_epochs:
                with torch.no_grad():
                    sim = cosine_sim_matrix(img_emb, txt_emb).cpu().numpy()[:16, :16]
                    sim_snapshots[epoch + 1] = sim
        
        avg_loss = epoch_loss / len(train_loader)
        tau = (1 / model.logit_scale.exp()).item()
        if (epoch + 1) % 3 == 0 or epoch == 0:
            print(f'Epoch {epoch+1}/{n_epochs}, Loss: {avg_loss:.4f}, '
                  f'τ={tau:.4f}, LR: {scheduler.get_last_lr()[0]:.2e}')
    
    return loss_history, sim_snapshots


# ======== 训练主模型 ========
set_seed(42)
model = MiniCLIP(vocab_size=tokenizer.vocab_size).to(device)
print(f'模型参数量: {count_params(model):,}\n')

loss_history, sim_snapshots = train_clip(
    model, train_loader, n_epochs=15, lr=5e-4, device=device,
    snapshot_epochs={1, 5, 10, 15}
)

In [ ]:
# ======== 可视化训练过程 ========

fig, axes = plt.subplots(1, 5, figsize=(22, 4))

# (a) Loss 曲线
window = 20
smoothed = np.convolve(loss_history, np.ones(window)/window, mode='valid')
axes[0].plot(loss_history, alpha=0.2, color='blue')
axes[0].plot(range(window-1, len(loss_history)), smoothed, color='red', linewidth=2)
axes[0].set_xlabel('训练步数')
axes[0].set_ylabel('Loss')
axes[0].set_title('(a) 训练 Loss 曲线')
axes[0].grid(True, alpha=0.3)

# (b-e) 相似度矩阵快照
subplot_labels = ['b', 'c', 'd', 'e']
for idx, epoch in enumerate(sorted(sim_snapshots.keys())):
    ax = axes[idx + 1]
    im = ax.imshow(sim_snapshots[epoch], cmap='Blues', vmin=-0.5, vmax=1)
    ax.set_title(f'({subplot_labels[idx]}) Epoch {epoch} 相似度')
    ax.set_xlabel('Text 索引')
    if idx == 0:
        ax.set_ylabel('Image 索引')
    plt.colorbar(im, ax=ax, shrink=0.8)

plt.suptitle('训练过程：Loss 下降 + 相似度矩阵对角线逐渐变亮', fontsize=14)
plt.tight_layout()
plt.show()

print(f'初始 Loss: {loss_history[0]:.4f}')
print(f'最终 Loss: {np.mean(loss_history[-20:]):.4f}')
print(f'最终温度 τ: {(1/model.logit_scale.exp()).item():.4f}')
print('\n观察：对角线（正样本对）的余弦相似度逐渐增大，off-diagonal（负样本对）逐渐减小')

## Part 7: 消融实验 A — 对称 vs 非对称损失

CLIP 使用**对称损失** $\mathcal{L} = \frac{1}{2}(\mathcal{L}_{i \to t} + \mathcal{L}_{t \to i})$，
同时优化两个方向的对齐。如果只优化其中一个方向会怎样？

- **Image→Text only** ($\mathcal{L}_{i \to t}$)：每张图找匹配的文本——图像表示被强约束，文本表示可能散乱
- **Text→Image only** ($\mathcal{L}_{t \to i}$)：每段文本找匹配的图像——文本表示被强约束，图像表示可能散乱
- **Symmetric**：两个方向同时优化，embedding 空间对齐更完整

下面我们训练三个模型，通过零样本分类准确率验证对称损失的优势。

In [ ]:
# ======== Part 7: 消融实验——对称 vs 非对称损失 ========

# 三种损失函数
def loss_i2t_only(logits):
    """只优化 Image→Text 方向"""
    labels = torch.arange(logits.size(0), device=logits.device)
    return F.cross_entropy(logits, labels)

def loss_t2i_only(logits):
    """只优化 Text→Image 方向"""
    labels = torch.arange(logits.size(0), device=logits.device)
    return F.cross_entropy(logits.T, labels)

def loss_symmetric(logits):
    """对称损失（两个方向取平均）"""
    return clip_loss(logits)


# 零样本分类函数（后续多处复用）
@torch.no_grad()
def zero_shot_accuracy(model, test_loader, tokenizer, templates=None, device='cpu'):
    """
    零样本分类：用文本 embedding 作为分类器权重
    
    Args:
        templates: 用于生成类名文本的模板列表，None 则使用 'a photo of a {}'
    Returns:
        accuracy (float)
    """
    model.eval()
    if templates is None:
        templates = ['a photo of a {}']
    
    # 编码所有类名文本（可用多模板 ensemble）
    class_embeddings = []
    for cls_name in CIFAR10_CLASSES:
        cls_embeds = []
        for template in templates:
            text = template.format(cls_name)
            tokens = torch.tensor([tokenizer.encode(text)], device=device)
            mask = (tokens != 0).float().to(device)
            embed = model.encode_text(tokens, mask)
            cls_embeds.append(embed)
        # 多模板平均 + 重新归一化
        avg_embed = torch.stack(cls_embeds).mean(dim=0)
        class_embeddings.append(F.normalize(avg_embed, dim=-1))
    
    class_embeddings = torch.cat(class_embeddings, dim=0)  # (10, proj_dim)
    
    correct = 0
    total = 0
    for images, _, _, labels in test_loader:
        images = images.to(device)
        img_embeds = model.encode_image(images)  # (B, proj_dim)
        
        # 余弦相似度 → 预测
        sims = cosine_sim_matrix(img_embeds, class_embeddings)  # (B, 10)
        preds = sims.argmax(dim=1).cpu()
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    
    model.train()
    return correct / total


# ======== 训练三个模型 ========
n_epochs_ablation = 10
ablation_losses = {}
ablation_accs = {}

loss_fns = {
    'Image→Text only': loss_i2t_only,
    'Text→Image only': loss_t2i_only,
    'Symmetric (CLIP)': loss_symmetric,
}

for name, loss_fn in loss_fns.items():
    print(f'\n{"="*50}')
    print(f'训练: {name}')
    print(f'{"="*50}')
    set_seed(42)
    ablation_model = MiniCLIP(vocab_size=tokenizer.vocab_size).to(device)
    losses, _ = train_clip(
        ablation_model, train_loader, n_epochs=n_epochs_ablation,
        lr=5e-4, device=device, loss_fn=loss_fn
    )
    ablation_losses[name] = losses
    acc = zero_shot_accuracy(ablation_model, test_loader, tokenizer, device=device)
    ablation_accs[name] = acc
    print(f'零样本准确率: {acc:.4f}')

# ======== 可视化 ========
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# (a) Loss 曲线
window = 20
colors = {'Image→Text only': 'blue', 'Text→Image only': 'orange', 'Symmetric (CLIP)': 'green'}
for name, losses in ablation_losses.items():
    smoothed = np.convolve(losses, np.ones(window)/window, mode='valid')
    axes[0].plot(smoothed, label=name, color=colors[name], linewidth=2)
axes[0].set_xlabel('训练步数')
axes[0].set_ylabel('Loss')
axes[0].set_title('(a) 不同损失方向的训练曲线')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# (b) 零样本准确率柱状图
names = list(ablation_accs.keys())
accs = list(ablation_accs.values())
bars = axes[1].bar(range(len(names)), accs, color=[colors[n] for n in names], alpha=0.8)
axes[1].set_xticks(range(len(names)))
axes[1].set_xticklabels(names, fontsize=9)
axes[1].set_ylabel('零样本分类准确率')
axes[1].set_title('(b) 对称 vs 非对称损失的零样本性能')
axes[1].axhline(y=0.1, color='red', linestyle='--', alpha=0.5, label='随机基线 (10%)')
axes[1].legend()
for bar, acc in zip(bars, accs):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.005,
                f'{acc:.1%}', ha='center', fontsize=10)

plt.tight_layout()
plt.show()

print('\n结论：')
print('- 对称损失同时约束两个方向的 embedding 对齐，通常表现最好')
print('- 单向损失只保证一个方向的检索质量，另一个方向可能退化')

## Part 8: 消融实验 B — Batch Size 对对比学习的影响

对比学习的核心在于**负样本数量**。InfoNCE 的互信息下界为：

$$I(X; Y) \geq \log K - \mathcal{L}_{\text{InfoNCE}}$$

$K$ = batch size，更大的 $K$ 意味着：
- 更紧的互信息下界（理论上可以捕获更多信息）
- 更多的负样本，让模型看到更多反例
- CLIP 原文使用 batch_size=32,768——我们测试 {32, 64, 128, 256} 观察趋势

In [ ]:
# ======== Part 8: 消融实验——Batch Size 对对比学习的影响 ========

batch_sizes = [32, 64, 128, 256]
bs_losses = {}
bs_accs = {}

for bs in batch_sizes:
    print(f'\n{"="*50}')
    print(f'训练: batch_size={bs} (负样本数 K={bs})')
    print(f'{"="*50}')
    
    # 创建对应 batch size 的 dataloader
    bs_loader = DataLoader(train_dataset, batch_size=bs, shuffle=True,
                           num_workers=2, drop_last=True)
    
    set_seed(42)
    bs_model = MiniCLIP(vocab_size=tokenizer.vocab_size).to(device)
    losses, _ = train_clip(
        bs_model, bs_loader, n_epochs=n_epochs_ablation,
        lr=5e-4, device=device
    )
    bs_losses[bs] = losses
    acc = zero_shot_accuracy(bs_model, test_loader, tokenizer, device=device)
    bs_accs[bs] = acc
    print(f'零样本准确率: {acc:.4f}, 理论 log(K)={np.log(bs):.2f}')

# ======== 可视化 ========
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# (a) Loss 曲线（按训练步数）
colors_bs = {32: 'red', 64: 'orange', 128: 'blue', 256: 'green'}
window = 15
for bs, losses in bs_losses.items():
    if len(losses) > window:
        smoothed = np.convolve(losses, np.ones(window)/window, mode='valid')
        axes[0].plot(smoothed, label=f'BS={bs}', color=colors_bs[bs], linewidth=2, alpha=0.8)
axes[0].set_xlabel('训练步数')
axes[0].set_ylabel('Loss')
axes[0].set_title('(a) 不同 Batch Size 的训练曲线')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# (b) 准确率 vs Batch Size
bs_list = sorted(bs_accs.keys())
acc_list = [bs_accs[bs] for bs in bs_list]
axes[1].plot(bs_list, acc_list, 'go-', linewidth=2, markersize=8)
for bs, acc in zip(bs_list, acc_list):
    axes[1].annotate(f'{acc:.1%}', (bs, acc), textcoords='offset points',
                     xytext=(0, 12), ha='center', fontsize=10)
axes[1].set_xlabel('Batch Size (= 负样本数 K)')
axes[1].set_ylabel('零样本分类准确率')
axes[1].set_title('(b) Batch Size 与零样本性能')
axes[1].axhline(y=0.1, color='red', linestyle='--', alpha=0.5, label='随机基线')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# 添加 log(K) 理论上界参考
ax2 = axes[1].twinx()
log_k = [np.log(bs) for bs in bs_list]
ax2.plot(bs_list, log_k, 'b--', alpha=0.5, linewidth=1)
ax2.set_ylabel('log(K) 互信息上界', color='blue', alpha=0.5)

plt.tight_layout()
plt.show()

print('\n结论：')
print('- 更大的 batch size 提供更多负样本，模型可以学到更好的判别性表示')
print('- CLIP 原文用 32,768 的 batch size，这也是为什么 CLIP 训练需要大量 GPU')
for bs in bs_list:
    print(f'  BS={bs}: 准确率={bs_accs[bs]:.1%}, log(K)={np.log(bs):.2f}')

## Part 9: Embedding 空间分析——Alignment & Uniformity

Wang & Isola (2020) 提出用两个指标衡量对比学习表示的质量：

**Alignment**（对齐性）：正样本对应该靠近

$$\ell_{\text{align}} = \mathbb{E}_{(x,y) \sim p_{\text{pos}}} \|f(x) - f(y)\|^2$$

**Uniformity**（均匀性）：所有 embedding 应均匀分布在单位超球面上

$$\ell_{\text{uniform}} = \log \mathbb{E}_{(x,y) \sim p_{\text{data}}} e^{-2\|f(x) - f(y)\|^2}$$

好的表示 = 低 Alignment（正对齐紧密）+ 低 Uniformity（分布均匀）。

我们对比随机初始化和训练后模型的这两个指标。

In [ ]:
# ======== Part 9: 嵌入空间分析——Alignment & Uniformity ========

@torch.no_grad()
def compute_alignment(img_embeds, txt_embeds):
    """Alignment: 正样本对之间的 L2 距离的平方的均值"""
    return (img_embeds - txt_embeds).pow(2).sum(dim=-1).mean().item()

@torch.no_grad()
def compute_uniformity(embeds, t=2):
    """Uniformity: log E[exp(-t * ||f(x)-f(y)||^2)]"""
    sq_dists = torch.cdist(embeds, embeds, p=2).pow(2)
    # 排除对角线（自身距离为0）
    mask = ~torch.eye(sq_dists.size(0), dtype=bool, device=sq_dists.device)
    return torch.log(torch.exp(-t * sq_dists[mask]).mean()).item()

@torch.no_grad()
def extract_embeddings(model, loader, device, n_samples=2000):
    """从数据集中提取 image 和 text embedding"""
    model.eval()
    img_embeds, txt_embeds, labels_list = [], [], []
    total = 0
    for images, tokens, text_mask, labels in loader:
        if total >= n_samples:
            break
        images = images.to(device)
        tokens = tokens.to(device)
        text_mask = text_mask.to(device)
        img_e = model.encode_image(images)
        txt_e = model.encode_text(tokens, text_mask)
        img_embeds.append(img_e.cpu())
        txt_embeds.append(txt_e.cpu())
        labels_list.append(labels)
        total += images.size(0)
    model.train()
    return (torch.cat(img_embeds)[:n_samples],
            torch.cat(txt_embeds)[:n_samples],
            torch.cat(labels_list)[:n_samples])

# ======== 提取 embedding ========
# 训练后模型
img_emb_trained, txt_emb_trained, labels_all = extract_embeddings(model, test_loader, device, 2000)

# 随机初始化模型
set_seed(123)
random_model = MiniCLIP(vocab_size=tokenizer.vocab_size).to(device)
img_emb_random, txt_emb_random, _ = extract_embeddings(random_model, test_loader, device, 2000)

# 计算 Alignment & Uniformity
align_trained = compute_alignment(img_emb_trained, txt_emb_trained)
uniform_img_trained = compute_uniformity(img_emb_trained[:500])
uniform_txt_trained = compute_uniformity(txt_emb_trained[:500])

align_random = compute_alignment(img_emb_random, txt_emb_random)
uniform_img_random = compute_uniformity(img_emb_random[:500])
uniform_txt_random = compute_uniformity(txt_emb_random[:500])

# ======== 三面板可视化 ========
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# (a) Alignment-Uniformity 散点图
ax = axes[0]
ax.scatter(uniform_img_random, align_random, s=100, marker='x', color='red', label='随机初始化', zorder=5)
ax.scatter(uniform_img_trained, align_trained, s=100, marker='o', color='green', label='训练后', zorder=5)
ax.set_xlabel('Uniformity ↓ (越低越好)')
ax.set_ylabel('Alignment ↓ (越低越好)')
ax.set_title('(a) Alignment vs Uniformity')
ax.legend()
ax.annotate('理想方向', xy=(min(uniform_img_random, uniform_img_trained) - 0.3,
            min(align_random, align_trained) - 0.1),
            fontsize=10, color='gray',
            arrowprops=dict(arrowstyle='->', color='gray'),
            xytext=(uniform_img_random, align_random))

# (b) 正样本 vs 负样本的余弦相似度分布
# 正样本: 匹配的 image-text 对
pos_sims = (img_emb_trained * txt_emb_trained).sum(dim=-1).numpy()
# 负样本: 不匹配的对（取前 2000 个 off-diagonal）
all_sims = cosine_sim_matrix(img_emb_trained[:200], txt_emb_trained[:200])
neg_mask = ~torch.eye(200, dtype=bool)
neg_sims = all_sims[neg_mask].numpy()

axes[1].hist(pos_sims, bins=30, alpha=0.7, label='正样本对', color='green', density=True)
axes[1].hist(neg_sims, bins=30, alpha=0.7, label='负样本对', color='red', density=True)
axes[1].set_xlabel('余弦相似度')
axes[1].set_ylabel('密度')
axes[1].set_title('(b) 正/负样本对的余弦相似度分布')
axes[1].legend()
axes[1].axvline(x=0, color='black', linestyle='--', alpha=0.3)

# (c) 类间相似度热力图（10×10）
class_img_embeds = []
class_txt_embeds = []
for c in range(10):
    mask_c = (labels_all == c)
    class_img_embeds.append(img_emb_trained[mask_c].mean(dim=0))
    # 用固定模板生成类文本 embedding
    text = f'a photo of a {CIFAR10_CLASSES[c]}'
    tokens_c = torch.tensor([tokenizer.encode(text)], device=device)
    mask_t = (tokens_c != 0).float().to(device)
    model.eval()
    with torch.no_grad():
        txt_e = model.encode_text(tokens_c, mask_t).cpu()
    class_txt_embeds.append(txt_e.squeeze())

class_img = F.normalize(torch.stack(class_img_embeds), dim=-1)
class_txt = F.normalize(torch.stack(class_txt_embeds), dim=-1)
class_sim = cosine_sim_matrix(class_img, class_txt).numpy()

im = axes[2].imshow(class_sim, cmap='RdYlGn', vmin=-0.5, vmax=1)
axes[2].set_xticks(range(10))
axes[2].set_yticks(range(10))
axes[2].set_xticklabels(CIFAR10_CLASSES, rotation=45, ha='right', fontsize=7)
axes[2].set_yticklabels(CIFAR10_CLASSES, fontsize=7)
axes[2].set_xlabel('Text 类别')
axes[2].set_ylabel('Image 类别')
axes[2].set_title('(c) 类间 Image-Text 余弦相似度')
plt.colorbar(im, ax=axes[2], shrink=0.8)

plt.suptitle('Embedding 空间质量分析', fontsize=14)
plt.tight_layout()
plt.show()

model.train()

# 验证
assert align_trained < align_random, '训练后 alignment 应优于随机'
print('✓ 训练后模型的 Alignment 优于随机初始化')

print(f'\n定量结果:')
print(f'  随机初始化: Alignment={align_random:.4f}, Uniformity(img)={uniform_img_random:.4f}')
print(f'  训练后:     Alignment={align_trained:.4f}, Uniformity(img)={uniform_img_trained:.4f}')
print(f'\n观察:')
print('- 训练后 Alignment ↓：正样本对的 embedding 更接近')
print('- 正负样本的余弦相似度分布分离良好 → 模型学到了有判别力的表示')
print('- 类间相似度矩阵对角线最亮 → image-text 按类别正确对齐')

## Part 10: 零样本分类 + Prompt Engineering 实验

CLIP 的零样本分类流程：
1. 为每个类别生成文本描述（如 "a photo of a cat"）
2. 编码所有类别文本 → 文本 embedding 矩阵 (C, D)
3. 编码查询图像 → 图像 embedding (1, D)
4. 计算余弦相似度，取 argmax 作为预测类别

**Prompt Engineering** 的核心发现（CLIP 论文 Table 5）：
- 单模板 `"a photo of a {}"` 比直接用类名 `"{}"` 好
- **80 个模板的 Ensemble** 比单模板再高 **+4.8%**

我们测试 4 种 prompt 策略，验证模板选择对零样本性能的影响。

In [ ]:
# ======== Part 10: 零样本分类与 Prompt Engineering 实验 ========

# 策略 1: 单模板（最简单）
strategy_single = ['a photo of a {}']

# 策略 2: 三模板
strategy_three = [
    'a photo of a {}',
    'a picture of a {}',
    'this is a {}',
]

# 策略 3: Prompt Ensemble（全部 12 个模板取平均）
strategy_ensemble = CAPTION_TEMPLATES

# 策略 4: 信息增强模板（加入类别语义上下文）
CATEGORY_CONTEXT = {
    'airplane': 'a type of aircraft',
    'automobile': 'a type of vehicle',
    'bird': 'a type of animal',
    'cat': 'a type of pet',
    'deer': 'a type of wildlife',
    'dog': 'a type of pet',
    'frog': 'a type of amphibian',
    'horse': 'a type of animal',
    'ship': 'a type of vessel',
    'truck': 'a type of vehicle',
}

@torch.no_grad()
def zero_shot_with_context(model, test_loader, tokenizer, device='cpu'):
    """使用带上下文信息的 prompt 做零样本分类"""
    model.eval()
    class_embeddings = []
    for cls_name in CIFAR10_CLASSES:
        context = CATEGORY_CONTEXT[cls_name]
        templates = [
            f'a photo of a {cls_name}, {context}',
            f'a {cls_name}, {context}',
            f'an image of a {cls_name}, {context}',
        ]
        cls_embeds = []
        for text in templates:
            tokens = torch.tensor([tokenizer.encode(text)], device=device)
            mask = (tokens != 0).float().to(device)
            embed = model.encode_text(tokens, mask)
            cls_embeds.append(embed)
        avg_embed = torch.stack(cls_embeds).mean(dim=0)
        class_embeddings.append(F.normalize(avg_embed, dim=-1))
    
    class_embeddings = torch.cat(class_embeddings, dim=0)
    
    correct = 0
    total = 0
    per_class_correct = torch.zeros(10)
    per_class_total = torch.zeros(10)
    for images, _, _, labels in test_loader:
        images = images.to(device)
        img_embeds = model.encode_image(images)
        sims = cosine_sim_matrix(img_embeds, class_embeddings)
        preds = sims.argmax(dim=1).cpu()
        correct += (preds == labels).sum().item()
        total += labels.size(0)
        for c in range(10):
            mask_c = (labels == c)
            per_class_correct[c] += (preds[mask_c] == labels[mask_c]).sum().item()
            per_class_total[c] += mask_c.sum().item()
    
    model.train()
    per_class_acc = per_class_correct / per_class_total.clamp(min=1)
    return correct / total, per_class_acc

# ======== 评估所有策略 ========
strategies = {
    '单模板': strategy_single,
    '三模板': strategy_three,
    'Ensemble (12模板)': strategy_ensemble,
}

prompt_accs = {}
for name, templates in strategies.items():
    acc = zero_shot_accuracy(model, test_loader, tokenizer, templates=templates, device=device)
    prompt_accs[name] = acc
    print(f'{name}: 准确率 = {acc:.4f}')

# 信息增强策略单独处理（不同接口）
context_acc, per_class_acc = zero_shot_with_context(model, test_loader, tokenizer, device=device)
prompt_accs['信息增强模板'] = context_acc
print(f'信息增强模板: 准确率 = {context_acc:.4f}')

# ======== 可视化 ========
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# (a) 策略对比柱状图
names = list(prompt_accs.keys())
accs = list(prompt_accs.values())
bars = axes[0].bar(range(len(names)), accs, color=['#3498db', '#2ecc71', '#e74c3c', '#9b59b6'], alpha=0.8)
axes[0].set_xticks(range(len(names)))
axes[0].set_xticklabels(names, fontsize=9)
axes[0].set_ylabel('零样本分类准确率')
axes[0].set_title('(a) Prompt Engineering 策略对比')
axes[0].axhline(y=0.1, color='gray', linestyle='--', alpha=0.5, label='随机基线')
axes[0].legend()
for bar, acc in zip(bars, accs):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.005,
                f'{acc:.1%}', ha='center', fontsize=10)

# (b) 逐类准确率（使用信息增强策略）
axes[1].barh(range(10), per_class_acc.numpy(), color='steelblue', alpha=0.8)
axes[1].set_yticks(range(10))
axes[1].set_yticklabels(CIFAR10_CLASSES)
axes[1].set_xlabel('准确率')
axes[1].set_title('(b) 逐类零样本准确率（信息增强模板）')
axes[1].axvline(x=0.1, color='red', linestyle='--', alpha=0.5, label='随机基线')
axes[1].legend()
for i, acc_i in enumerate(per_class_acc):
    axes[1].text(acc_i + 0.01, i, f'{acc_i:.0%}', va='center', fontsize=9)

plt.tight_layout()
plt.show()

print('\n结论：')
print('- Prompt Ensemble 通过平均多个模板的 text embedding 减少了单一模板的偏差')
print('- 信息增强模板加入类别上下文（如 "a type of vehicle"），提供额外语义信号')
print('- 这验证了 CLIP 论文的发现：prompt 工程对零样本性能有显著影响')

## Part 11: SigLIP — Sigmoid 替代 Softmax

SigLIP (Zhai et al., 2023) 将 InfoNCE 的 N-way softmax 分类重新表述为 $N^2$ 个独立的二分类问题：

$$\mathcal{L}_{\text{SigLIP}} = -\frac{1}{N^2} \sum_{i,j} \log \sigma(y_{ij} \cdot (s_{ij} \cdot e^\tau + b))$$

其中 $y_{ij} = 1$ 当 $i=j$（正样本对），$y_{ij} = -1$ 当 $i \neq j$（负样本对）。

**SigLIP 的优势**：
- 无需全局 softmax 归一化（InfoNCE 需要整个 batch 的 logits）
- 分布式训练更友好（不需要 all-gather 操作）
- 对假阳性（同 batch 中不同图片但相同语义）更鲁棒

下面我们实现 SigLIP 损失，与 InfoNCE 在相同条件下对比。

In [ ]:
# ======== Part 11: SigLIP 损失——Sigmoid 替代 Softmax ========

def siglip_loss(logits):
    """
    SigLIP 损失：将 N-way 分类转为 N^2 个二分类
    
    logits: (N, N) 已经过温度缩放的相似度矩阵
    """
    N = logits.size(0)
    
    # 标签矩阵: 对角线=1 (正样本), 其余=-1 (负样本)
    labels = 2 * torch.eye(N, device=logits.device) - 1  # (N, N)
    
    # 添加可学习 bias（简化处理：使用固定 bias=-10）
    # SigLIP 论文使用可学习 bias，这里为简化固定
    bias = -10.0
    
    # Sigmoid 二分类损失: -log(sigmoid(y * (s + b)))
    # 等价于: log(1 + exp(-y * (s + b)))
    loss = -F.logsigmoid(labels * (logits + bias))
    
    return loss.mean()


# ======== 验证 SigLIP 损失 ========
set_seed(42)
test_logits = torch.randn(4, 4)
loss_val = siglip_loss(test_logits)
print(f'SigLIP 损失值 (随机 4x4): {loss_val.item():.4f}')

# 理想情况验证：完美对齐时损失应该很低
perfect_logits = torch.eye(4) * 20 - 10  # 对角线 +10, 其余 -10
perfect_loss = siglip_loss(perfect_logits)
print(f'SigLIP 损失值 (完美对齐): {perfect_loss.item():.6f} (应接近 0)')
assert perfect_loss.item() < 0.01, 'SigLIP 在完美对齐时损失应接近 0'
print('✓ SigLIP 损失验证通过')


# ======== 包装为可用于 train_clip 的损失函数 ========
def siglip_loss_fn(logits):
    """适配 train_clip 接口的 SigLIP 损失"""
    return siglip_loss(logits)


# ======== InfoNCE vs SigLIP 对比训练 ========
n_epochs_compare = 10
print(f'\n{"="*50}')
print('对比训练: InfoNCE vs SigLIP')
print(f'{"="*50}')

# InfoNCE 模型
print('\n--- InfoNCE ---')
set_seed(42)
model_infonce = MiniCLIP(vocab_size=tokenizer.vocab_size).to(device)
losses_infonce, _ = train_clip(
    model_infonce, train_loader, n_epochs=n_epochs_compare,
    lr=5e-4, device=device, loss_fn=clip_loss
)
acc_infonce = zero_shot_accuracy(model_infonce, test_loader, tokenizer, device=device)

# SigLIP 模型
print('\n--- SigLIP ---')
set_seed(42)
model_siglip = MiniCLIP(vocab_size=tokenizer.vocab_size).to(device)
losses_siglip, _ = train_clip(
    model_siglip, train_loader, n_epochs=n_epochs_compare,
    lr=5e-4, device=device, loss_fn=siglip_loss_fn
)
acc_siglip = zero_shot_accuracy(model_siglip, test_loader, tokenizer, device=device)

print(f'\nInfoNCE 零样本准确率: {acc_infonce:.4f}')
print(f'SigLIP  零样本准确率: {acc_siglip:.4f}')

# ======== 可视化对比 ========
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# (a) Loss 曲线（归一化到初始值 = 1）
window = 20
for name, losses, color in [('InfoNCE', losses_infonce, 'blue'), ('SigLIP', losses_siglip, 'orange')]:
    initial = losses[0]
    normalized = [l / initial for l in losses]
    if len(normalized) > window:
        smoothed = np.convolve(normalized, np.ones(window)/window, mode='valid')
        axes[0].plot(smoothed, label=name, color=color, linewidth=2)
axes[0].set_xlabel('训练步数')
axes[0].set_ylabel('归一化 Loss (初始=1)')
axes[0].set_title('(a) InfoNCE vs SigLIP 训练曲线')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# (b) 零样本准确率对比
bars = axes[1].bar(['InfoNCE', 'SigLIP'], [acc_infonce, acc_siglip],
                    color=['blue', 'orange'], alpha=0.8)
axes[1].set_ylabel('零样本分类准确率')
axes[1].set_title('(b) 零样本分类性能')
axes[1].axhline(y=0.1, color='red', linestyle='--', alpha=0.5, label='随机基线')
axes[1].legend()
for bar, acc in zip(bars, [acc_infonce, acc_siglip]):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.005,
                f'{acc:.1%}', ha='center', fontsize=11)

# (c) 相似度矩阵对比
model_infonce.eval()
model_siglip.eval()
with torch.no_grad():
    sample_imgs = next(iter(test_loader))[0][:16].to(device)
    sample_tokens = next(iter(test_loader))[1][:16].to(device)
    sample_mask = next(iter(test_loader))[2][:16].to(device)
    
    img_e_i = model_infonce.encode_image(sample_imgs)
    txt_e_i = model_infonce.encode_text(sample_tokens, sample_mask)
    sim_infonce = cosine_sim_matrix(img_e_i, txt_e_i).cpu().numpy()
    
    img_e_s = model_siglip.encode_image(sample_imgs)
    txt_e_s = model_siglip.encode_text(sample_tokens, sample_mask)
    sim_siglip = cosine_sim_matrix(img_e_s, txt_e_s).cpu().numpy()

# 显示两个矩阵的差异
diff_sim = sim_infonce - sim_siglip
im = axes[2].imshow(diff_sim, cmap='RdBu_r', vmin=-0.5, vmax=0.5)
axes[2].set_title('(c) 相似度差异 (InfoNCE - SigLIP)')
axes[2].set_xlabel('Text 索引')
axes[2].set_ylabel('Image 索引')
plt.colorbar(im, ax=axes[2], shrink=0.8)

model_infonce.train()
model_siglip.train()

plt.suptitle('InfoNCE vs SigLIP 全面对比', fontsize=14)
plt.tight_layout()
plt.show()

print('\n结论：')
print('- SigLIP 和 InfoNCE 都能学到有意义的表示')
print('- SigLIP 不需要全局 softmax，在分布式训练中更高效')
print('- 在小 batch size 下两者性能可能接近，SigLIP 的优势在大规模训练时更明显')

## Part 12: 图文检索——Recall@K 评估

对比学习模型天然支持检索任务：
- **Image→Text 检索**：给定一张图，找最匹配的文本描述
- **Text→Image 检索**：给定一段文本，找最匹配的图像

评估指标 **Recall@K**：在返回的 top-K 结果中，正确答案出现的比例。

$$\text{Recall@K} = \frac{1}{N} \sum_{i=1}^{N} \mathbb{1}[\text{正确答案} \in \text{top-K}(i)]$$

In [ ]:
# ======== Part 12: 图文检索——Recall@K 评估 ========

@torch.no_grad()
def compute_retrieval_metrics(model, test_loader, tokenizer, device, n_samples=1000):
    """
    计算图文检索的 Recall@K
    
    注意：CIFAR-10 的 caption 是模板生成的，同一类别的 caption 语义等价，
    因此"正确"的定义是检索结果与查询属于同一类别。
    """
    model.eval()
    img_embeds, txt_embeds, all_labels = [], [], []
    total = 0
    
    for images, tokens, text_mask, labels in test_loader:
        if total >= n_samples:
            break
        images = images.to(device)
        tokens = tokens.to(device)
        text_mask = text_mask.to(device)
        
        img_e = model.encode_image(images)
        txt_e = model.encode_text(tokens, text_mask)
        
        img_embeds.append(img_e.cpu())
        txt_embeds.append(txt_e.cpu())
        all_labels.append(labels)
        total += images.size(0)
    
    img_embeds = torch.cat(img_embeds)[:n_samples]
    txt_embeds = torch.cat(txt_embeds)[:n_samples]
    all_labels = torch.cat(all_labels)[:n_samples]
    
    # 相似度矩阵
    sim = cosine_sim_matrix(img_embeds, txt_embeds)  # (N, N)
    
    # 相关性矩阵: 同类别即为正确匹配
    relevance = (all_labels.unsqueeze(1) == all_labels.unsqueeze(0)).float()  # (N, N)
    
    results = {}
    for k in [1, 5, 10]:
        # Image → Text
        _, topk_i2t = sim.topk(k, dim=1)  # (N, K)
        hit_i2t = relevance.gather(1, topk_i2t).sum(dim=1).clamp(max=1)  # 是否命中
        recall_i2t = hit_i2t.mean().item()
        
        # Text → Image
        _, topk_t2i = sim.T.topk(k, dim=1)
        hit_t2i = relevance.T.gather(1, topk_t2i).sum(dim=1).clamp(max=1)
        recall_t2i = hit_t2i.mean().item()
        
        results[f'I→T R@{k}'] = recall_i2t
        results[f'T→I R@{k}'] = recall_t2i
    
    model.train()
    return results


# ======== 评估训练后的主模型 ========
retrieval_results = compute_retrieval_metrics(model, test_loader, tokenizer, device, n_samples=1000)

print('图文检索 Recall@K:')
print(f'{"指标":<12} {"值":>8}')
print('-' * 22)
for metric, value in retrieval_results.items():
    print(f'{metric:<12} {value:>8.1%}')

# ======== 可视化检索结果 ========
model.eval()

# 取 5 张查询图片
n_queries = 5
n_results = 5

fig, axes = plt.subplots(n_queries, n_results + 1, figsize=(3 * (n_results + 1), 3 * n_queries))
fig.suptitle('Image→Text 检索示例（绿色=同类别，红色=不同类别）', fontsize=14)

with torch.no_grad():
    # 提取一批 embedding
    batch = next(iter(test_loader))
    images_vis = batch[0][:50].to(device)
    tokens_vis = batch[1][:50].to(device)
    mask_vis = batch[2][:50].to(device)
    labels_vis = batch[3][:50]
    
    img_e = model.encode_image(images_vis)
    txt_e = model.encode_text(tokens_vis, mask_vis)
    sims = cosine_sim_matrix(img_e, txt_e).cpu()

for q in range(n_queries):
    # 显示查询图片
    query_img = inv_norm(images_vis[q].cpu()).permute(1, 2, 0).clamp(0, 1).numpy()
    axes[q, 0].imshow(query_img)
    axes[q, 0].set_title(f'Query: {CIFAR10_CLASSES[labels_vis[q]]}', fontsize=9, fontweight='bold')
    axes[q, 0].axis('off')
    
    # Top-K 检索结果
    _, topk = sims[q].topk(n_results)
    for r, idx in enumerate(topk):
        result_img = inv_norm(images_vis[idx].cpu()).permute(1, 2, 0).clamp(0, 1).numpy()
        axes[q, r + 1].imshow(result_img)
        is_correct = labels_vis[q] == labels_vis[idx]
        color = 'green' if is_correct else 'red'
        caption = tokenizer.decode(tokens_vis[idx].cpu().tolist())[:15]
        axes[q, r + 1].set_title(f'{caption}...\nsim={sims[q, idx]:.2f}',
                                  fontsize=7, color=color)
        axes[q, r + 1].axis('off')
        for spine in axes[q, r + 1].spines.values():
            spine.set_edgecolor(color)
            spine.set_linewidth(3)

plt.tight_layout()
plt.show()
model.train()

# 验证
assert retrieval_results['I→T R@10'] > retrieval_results['I→T R@1'], 'R@10 应 > R@1'
print('\n✓ Recall@K 单调递增验证通过')
print('\n观察：')
print('- 同类别的图文对在 embedding 空间中距离更近')
print('- Recall@K 随 K 增大而提高，说明正确答案通常在候选列表的前部')

## Part 13: 实验结论

### 核心发现

| # | 实验 | 发现 |
|---|------|------|
| 1 | InfoNCE 梯度分析 | 梯度集中在正样本对（负梯度→拉近）和困难负样本（正梯度→推远） |
| 2 | 温度参数 τ | τ≈0.07 在 softmax 锐度和梯度稳定性之间取得最佳平衡 |
| 3 | 相似度矩阵演化 | 训练过程中对角线逐渐变亮——正样本对相似度增大，负样本对减小 |
| 4 | 对称 vs 非对称损失 | 对称损失同时约束两个方向，通常优于任何单向损失 |
| 5 | Batch Size 影响 | 更大 batch = 更多负样本 = 更紧的互信息下界 = 更好的性能 |
| 6 | Alignment & Uniformity | 训练后 Alignment↓ + Uniformity↓，正负样本余弦相似度分离良好 |
| 7 | Prompt Engineering | 多模板 Ensemble 优于单模板，验证了 CLIP 论文的 +4.8% 发现 |
| 8 | SigLIP vs InfoNCE | SigLIP 用 sigmoid 替代 softmax，无需全局归一化，性能可比 |

### MiniCLIP vs 真实 CLIP

| 维度 | 我们的 MiniCLIP | 真实 CLIP (ViT-L/14) |
|------|:---:|:---:|
| 参数量 | ~3M | 428M |
| Vision Encoder | 4 层, d=192 | 24 层, d=1024 |
| Text Encoder | 3 层, d=192 | 12 层, d=512 |
| 投影维度 | 128 | 768 |
| 训练数据 | CIFAR-10 (50K) | WIT-400M (4亿) |
| Batch Size | 128 | 32,768 |
| 训练时间 | 分钟 (CPU) | 数天 (256-1024 GPU) |
| Tokenizer | 字符级 (~97) | BPE (~49K) |
| Zero-Shot ImageNet | N/A | 76.2% |
| 温度 τ | 可学习 (初始 0.07) | 可学习 (初始 0.07) ✓ |
| 损失函数 | 对称 InfoNCE | 对称 InfoNCE ✓ |

### 与理论笔记的对照

本实验验证了以下理论概念：

- **InfoNCE = N-way Cross-Entropy** → Part 2 的数值验证证实了[对比学习笔记](../../notes/fundamentals/contrastive-learning.md)中的公式推导
- **温度参数的 hard negative mining 效应** → Part 3 可视化对应笔记中的温度分析表格
- **CLIP 的对称损失设计** → Part 7 消融实验验证了 [CLIP 论文笔记](../../papers/clip.md)中的设计选择
- **Batch Size 与互信息下界** → Part 8 实验对应 $I(X;Y) \geq \log K - \mathcal{L}$
- **SigLIP 的分布式训练优势** → Part 11 实现对应[对比学习笔记](../../notes/fundamentals/contrastive-learning.md)中 SigLIP 章节

### 后续实验建议

- 使用真实 caption 数据集（Flickr30k, COCO Captions）替代模板生成
- 实现 MoCo 风格的 momentum encoder + queue，对比不同负样本策略
- 增加线性探测（Linear Probe）评估，对比零样本与有监督微调
- 使用 GradCAM/Attention Rollout 可视化 ViT 关注的图像区域
- 实现 EVA-CLIP 的训练效率优化（masked image modeling 预训练）
- 尝试更大的模型规模，验证 Scaling Law 在对比学习中的适用性